In [1]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Function to load and preprocess images
def load_images_from_paths(df, img_size=(48, 48)):
    images = []
    labels = df['label'].values

    for img_path in df['image']:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        if img is not None:
            img = cv2.resize(img, img_size)  # Resize to 48x48
            img = img / 255.0  # Normalize pixel values to [0, 1]
            images.append(img)
        else:
            print(f"Failed to load image: {img_path}")

    # Convert to numpy arrays
    images = np.array(images).reshape(-1, img_size[0], img_size[1], 1)

    return images, labels

# Load training and testing images
X_train, y_train = load_images_from_paths(train)
X_test, y_test = load_images_from_paths(test)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


NameError: name 'train' is not defined

In [7]:
!pip install imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 4.4 MB/s eta 0:00:00


In [3]:
import os
import random
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Add, Conv2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import layers, Input
import tensorflow as tf
from tensorflow.keras import mixed_precision
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

100%|██████████| 60.3M/60.3M [00:00<00:00, 124MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/msambare/fer2013/versions/1


In [4]:
tr = path+"/train"
tet = path+"/test"

In [5]:
train_datagen = ImageDataGenerator(
    width_shift_range = 0.1,        # Randomly shift the width of images by up to 10%
    height_shift_range = 0.1,       # Randomly shift the height of images by up to 10%
    horizontal_flip = True,         # Flip images horizontally at random
    rescale = 1./255,               # Rescale pixel values to be between 0 and 1
    validation_split = 0.2          # Set aside 20% of the data for validation
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255,               # Rescale pixel values to be between 0 and 1
    validation_split = 0.2          # Set aside 20% of the data for validation
)

In [6]:
def residual_block(input_tensor, filters, kernel_size=3, stride=1):
    x = Conv2D(filters, kernel_size, strides=stride, padding='same', activation='relu')(input_tensor)
    x = BatchNormalization()(x)
    x = Conv2D(filters, kernel_size, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)

    shortcut = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(input_tensor) if stride != 1 else input_tensor
    shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

In [8]:
import os

def load_dataset(directory):
    image_paths = []
    labels = []

    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)  # Join paths correctly
        if not os.path.isdir(label_path):
            continue  # Skip if it's not a directory

        for filename in os.listdir(label_path):
            image_path = os.path.join(label_path, filename)
            image_paths.append(image_path)
            labels.append(label)

        print(label, "Completed")

    return image_paths, labels

In [9]:

train = pd.DataFrame()
train['image'], train['label'] = load_dataset(tr)
# shuffle the dataset
train = train.sample(frac=1).reset_index(drop=True)
train.head()

surprise Completed
angry Completed
neutral Completed
happy Completed
disgust Completed
fear Completed
sad Completed


,image,label
0,/root/.cache/kagglehub/datasets/msambare/fer20...,neutral
1,/root/.cache/kagglehub/datasets/msambare/fer20...,angry
2,/root/.cache/kagglehub/datasets/msambare/fer20...,neutral
3,/root/.cache/kagglehub/datasets/msambare/fer20...,neutral
4,/root/.cache/kagglehub/datasets/msambare/fer20...,sad


In [10]:
test = pd.DataFrame()
test['image'], test['label'] = load_dataset(tet)
test.head()

surprise Completed
angry Completed
neutral Completed
happy Completed
disgust Completed
fear Completed
sad Completed


,image,label
0,/root/.cache/kagglehub/datasets/msambare/fer20...,surprise
1,/root/.cache/kagglehub/datasets/msambare/fer20...,surprise
2,/root/.cache/kagglehub/datasets/msambare/fer20...,surprise
3,/root/.cache/kagglehub/datasets/msambare/fer20...,surprise
4,/root/.cache/kagglehub/datasets/msambare/fer20...,surprise


In [11]:

# Extract images and labels
X_train = np.stack(train['image'].values)  # Convert list of images to numpy array
y_train = train['label'].values

X_test = np.stack(test['image'].values)  # Convert list of images to numpy array
y_test = test['label'].values

In [28]:
import pandas as pd
import os
import cv2
import numpy as np

def load_dataset(df, img_size=(48, 48)):
    """
    Loads images from file paths in a DataFrame and preprocesses them.

    Args:
        df (pd.DataFrame): DataFrame containing 'image' column with file paths.
        img_size (tuple): Target image size for resizing.

    Returns:
        tuple: A tuple containing the list of images and the list of labels.
    """
    images = []
    labels = []  # Assuming 'label' column exists in df

    for index, row in df.iterrows():  # Iterate through DataFrame rows
        img_path = row['image']
        label = row['label']  # Assuming 'label' column exists
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale

        if img is not None:
            img = cv2.resize(img, img_size)  # Resize to 48x48
            img = img / 255.0  # Normalize pixel values to [0, 1]
            images.append(img)
            labels.append(label)
        else:
            print(f"Failed to load image: {img_path}")

    return images, labels

# Assuming 'train' is a DataFrame containing image file paths and labels
test = pd.DataFrame()
# Load the data
test['image'], test['label'] = load_dataset(tet) # Ensure 'tet' is replaced with the intended variable

AttributeError: 'str' object has no attribute 'iterrows'

In [44]:
print(X_train[0:10])

['/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1/train/happy/Training_89428402.jpg'
 '/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1/train/happy/Training_21969084.jpg'
 '/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1/train/happy/Training_58137516.jpg'
 '/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1/train/happy/Training_8372977.jpg'
 '/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1/train/sad/Training_39660363.jpg'
 '/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1/train/neutral/Training_59285594.jpg'
 '/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1/train/fear/Training_89031934.jpg'
 '/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1/train/fear/Training_80807753.jpg'
 '/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1/train/happy/Training_48577521.jpg'
 '/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1/train/sad/Training_58301844.jpg']


In [12]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Function to load and preprocess images
def load_images_from_paths(image_paths, img_size=(48, 48)):
    images = []

    for img_path in image_paths:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        if img is not None:
            img = cv2.resize(img, img_size)  # Resize to 48x48
            img = img.astype('float32')  # Ensure the data type is float32
            img = img / 255.0  # Normalize pixel values to [0, 1]
            img = np.expand_dims(img, axis=-1)  # Add an extra dimension for channels (grayscale)
            images.append(img)
        else:
            print(f"Failed to load image: {img_path}")

    # Convert to numpy array
    images = np.array(images)

    return images

# Assuming `X_train` contains the list of image file paths
X_train_images = load_images_from_paths(X_train)

# Check the shape of X_train_images
print(f"Shape of X_train_images: {X_train_images.shape}")


Shape of X_train_images: (28709, 48, 48, 1)


In [13]:
# Encode the labels if they are categorical
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, InputLayer
from tensorflow.keras.optimizers import Adam

# Define CNN model for classification

input_shape = (48, 48, 1)  # Assuming 48x48 grayscale images
cnn = Sequential([
    InputLayer(input_shape=(48, 48, 1)),  # Input layer for grayscale images of size 48x48
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(7, activation='softmax')  # Assuming 7 classes, adjust for your number of classes
])


cnn.summary()
# Compile the model
cnn.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
print(X_train.shape)
cnn.fit(X_train_images, y_train_encoded, epochs=10, batch_size=32)



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 48, 48, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 24, 24, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 24, 24, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 12, 12, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 683,527 (2.61 MB)

 Trainable params: 683,527 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

(28709,)
Epoch 1/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 159s 175ms/step - accuracy: 0.2924 - loss: 1.7280
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 197s 170ms/step - accuracy: 0.4716 - loss: 1.3693
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 157s 175ms/step - accuracy: 0.5396 - loss: 1.2128
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 201s 173ms/step - accuracy: 0.5949 - loss: 1.0833
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 156s 174ms/step - accuracy: 0.6321 - loss: 0.9865
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 200s 172ms/step - accuracy: 0.6749 - loss: 0.8815
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 202s 171ms/step - accuracy: 0.7167 - loss: 0.7672
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 154s 172ms/step - accuracy: 0.7654 - loss: 0.6476
Epoch 9/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 205s 176ms/step - accuracy: 0.8098 - loss: 0.5311
Epoch 10/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 198s 172ms/step - accuracy: 0.8476 - loss: 0.4252


In [15]:
# Save the trained model
model_save_path = '/content/baby.h5'  # Specify the path where you want to save the model
cnn.save(model_save_path)

print(f"Model saved to {model_save_path}")

Model saved to /content/baby.h5


In [16]:
X_test_images = load_images_from_paths(X_test)


In [19]:

# Evaluate the CNN model on the test set
accuracy = cnn.evaluate(X_test_images, y_test_encoded)




225/225 ━━━━━━━━━━━━━━━━━━━━ 12s 52ms/step - accuracy: 0.6180 - loss: 1.4781


In [ ]:
# Extract features from training and testing data
X_train_features = cnn.predict(X_train)
X_test_features = cnn.predict(X_test)

# Standardize the features
scaler = StandardScaler()
X_train_features = scaler.fit_transform(X_train_features)
X_test_features = scaler.transform(X_test_features)


898/898 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [ ]:
from sklearn.svm import SVC

# Train an SVM with the extracted features
svm = SVC(kernel='linear')  # You can change kernel to 'rbf', 'poly', etc.
svm.fit(X_train_features, y_train)

# Predict using the SVM classifier
y_pred = svm.predict(X_test_features)


In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

# Grid search
grid = GridSearchCV(SVC(), param_grid, cv=5)
grid.fit(X_train_features, y_train)

# Best SVM model
best_svm = grid.best_estimator_

# Predict and calculate accuracy with the best model
y_pred_best = best_svm.predict(X_test_features)
accuracy_best = accuracy_score(y_test, y_pred_best)
print("Best SVM Test Accuracy:", accuracy_best)
